In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
indicators=pd.read_csv("RHoMISmother_indicators.csv")
codebook=pd.read_csv("raw data code book.csv")
raw=pd.read_csv("RHoMISmother_rawdata.csv",encoding = "ISO-8859-1", low_memory=False)

In [4]:
# looking for duds in the HFIAS data
raw['HHid']=raw['HouseholdID']
HFIAS_data=pd.merge(raw[['HHid','HFIAS_1','HFIAS_2','HFIAS_3','HFIAS_4','HFIAS_5','HFIAS_6','HFIAS_7','HFIAS_8','HFIAS_9']],indicators[['HHid','HFIAS_status']],on='HHid')
HFIAS_most_null=[]
for i in range(len(HFIAS_data)):
    n=0
    for j in range(9):
        if pd.isnull( HFIAS_data.iloc[i,j+1] ):
            n+=1
    if n>=7:
        HFIAS_most_null.append(HFIAS_data.loc[i,'HHid'])
# create a numerical column for HFIAS status
# and this column will remove the really shit ones from the list of shit ones
# and a new column for income_percap
prob=0
for i in range(len(indicators)):
    if indicators.loc[i,'HHid'] in HFIAS_most_null:
        indicators.loc[i,'HFIAS_code']=None
    elif str(indicators.loc[i,'HFIAS_status'])=='FoodSecure':
        indicators.loc[i,'HFIAS_code']=0
    elif str(indicators.loc[i,'HFIAS_status'])=='MildlyFI':
        indicators.loc[i,'HFIAS_code']=1
    elif str(indicators.loc[i,'HFIAS_status'])=='ModeratelyFI':
        indicators.loc[i,'HFIAS_code']=2
    elif str(indicators.loc[i,'HFIAS_status'])=='SeverelyFI':
        indicators.loc[i,'HFIAS_code']=3
        continue
    else:
        prob+=1
zero=0
dz=0
for i in range(len(indicators)):
    if indicators.loc[(i,'HHsizeMAE')]!=0:
        indicators.loc[(i,'income_percap')]=np.divide(indicators.loc[(i,'total_income')],indicators.loc[(i,'HHsizeMAE')])
    else: 
        zero+=1
        if indicators.loc[(i,'HHsizemembers')]==0:
            dz+=1
            indicators.loc[(i,'income_percap')]=indicators.loc[(i,'total_income')]
        else:
            indicators.loc[(i,'income_percap')]=np.divide(indicators.loc[(i,'total_income')],indicators.loc[(i,'HHsizemembers')])
# make a new income percentile column
percentiles=np.zeros(10)
maps=np.arange(0,10,1)
for i in range(9):
    j=np.divide(i+1,10)
    percentiles[i+1]=indicators['income_percap'].quantile (q=j, interpolation='linear')
print(percentiles,maps)
indicators['inc_pc_perc']=None
for m in range(len(indicators)):
    k=9
    while indicators.loc[(m,'income_percap')]<percentiles[k]:
        k-=1
    indicators.loc[(m,'inc_pc_perc')]=maps[k]
indicators['inc_pc_perc']=pd.DataFrame(indicators['inc_pc_perc'],dtype='float')
print('There were problems in ',prob,' rows.')
print('There were ',zero,' households with no MAE. Of these, ',dz,' were recorded with no members at all.')

[  0.00000000e+00   1.84342857e-01   1.05003592e+01   2.58949485e+01
   5.14872286e+01   9.41080196e+01   1.56888886e+02   2.67285105e+02
   4.76762123e+02   1.09783781e+03] [0 1 2 3 4 5 6 7 8 9]
There were problems in  0  rows.
There were  36  households with no MAE. Of these,  15  were recorded with no members at all.


In [5]:
# make a new column
#for each country in raw data,
#for each region,
#for each village,
#restrict raw data to that village
#for each household id,
#take crop names as a small data frame
#append all of them
#village crop div = len(that.unique())-1 because of NaNs
#fill that into the new column

In [25]:
def crd(vil):
    # vil is a restricted data frame
    # finds the number of different crops grown in vil
    c=[]
    for i in range(len(vil)):
        for j in range(8):
            c.append(vil.loc[i,'crop_name_'+str(j+1)])
    crops=set(c)
    return crops

In [76]:
# BUT HOW IS CROPDIV CALCULATED???
indic=indicators[['HHid','Country','Region','Village','CropDiv','score_HDDSBadSeason','NrofMonthsFoodInsecure','income_percap','HFIAS_code']]
cols=['HHid']
for i in range(8):
    cols.append('crop_name_'+str(i+1))
rawd=raw[cols]
vilcropdata=pd.concat([indic,rawd])

In [77]:
vilcropdata

,Country,CropDiv,HFIAS_code,HHid,NrofMonthsFoodInsecure,Region,Village,crop_name_1,crop_name_2,crop_name_3,crop_name_4,crop_name_5,crop_name_6,crop_name_7,crop_name_8,income_percap,score_HDDSBadSeason
0,Kenya,3.0,2.0,1,5.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,4.0
1,Kenya,5.0,3.0,2,7.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.479876,3.0
2,Kenya,3.0,1.0,3,4.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1861.558219,7.0
3,Kenya,4.0,3.0,4,4.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.712096,4.0
4,Kenya,6.0,1.0,6,2.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.130597,3.0
5,Kenya,5.0,3.0,9,6.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.883895,5.0
6,Kenya,7.0,3.0,10,8.0,Wote,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.872024,5.0
7,Kenya,4.0,3.0,11,5.0,Wote,446.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.187050,4.0
8,Kenya,4.0,3.0,14,3.0,Wote,272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.0
9,Kenya,6.0,3.0,15,4.0,Wote,272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209.772390,6.0


In [64]:
def iteratevil():
    vilcropdiv=pd.con
    for a in indicators['Country'].unique():
        countrydf=indicators[indicators['Country']==a]
        for b in countrydf['Region'].unique():
            regiondf=countrydf[countrydf['Region']==b]
            for c in regiondf['Village'].unique():
                villagedf=regiondf[regiondf['Village']==c]
                hhds=list(villagedf['HHid'])
                print(hhds)
                print('-'*100)
                cr=[]
                for i in hhds:
                    for j in range(8):
                        
                vil=raw['HHid' == hhds]
                print(vil['HHid'])

In [65]:
iteratevil()

[1, 2, 3, 4, 6, 9, 10]
----------------------------------------------------------------------------------------------------


KeyError: False

In [31]:
first=raw.iloc[0:200,:]
b=crd(first)
b

{nan,
 'pigeon_pea',
 'beans',
 'sorghum',
 'tomato',
 'ground_nut',
 'other2',
 'cassava',
 'potato',
 'fodder_grass',
 'banana',
 'other3',
 'green_grams',
 'other1',
 'maize',
 'onion',
 'mango',
 'oranges',
 'millet',
 'cowpea'}

In [19]:
indicators.loc[0:2,'CropDiv']

0    3.0
1    5.0
2    3.0
Name: CropDiv, dtype: float64